In [1]:
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten ,Activation ,Dropout
from keras.callbacks import ReduceLROnPlateau, EarlyStopping , ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import VGG16
from keras.models import Sequential
from keras.optimizers import Adam
import matplotlib.pyplot as plt
import numpy as np
import keras
import os

Using TensorFlow backend.


### Read train and validation data using data generator


In [2]:
data_gen = ImageDataGenerator(rescale=1.0/255.0)
train_data = data_gen.flow_from_directory(directory="/home/hussein/Scene_Recognition/train",target_size=(256,256))
validation_data = data_gen.flow_from_directory(directory="/home/hussein/Scene_Recognition/validation", target_size=(256,256))


Found 4800 images belonging to 10 classes.
Found 1200 images belonging to 10 classes.


### Fine tune pre-trained vgg16

In [3]:
vgg_conv = VGG16(weights='vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5',
                  include_top=False,input_shape=(256,256,3))

## Disabling training from VGG layers
vgg_conv.trainable=False

transfer_model = Sequential()
transfer_model.add(vgg_conv)
transfer_model.add(Flatten())
transfer_model.add(Dropout(0.25))
transfer_model.add(Dense(64, activation='relu'))
transfer_model.add(Dropout(0.25))
transfer_model.add(Dense(10, activation='softmax'))

transfer_model.summary()


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 8, 8, 512)         14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 32768)             0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 32768)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                2097216   
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                650       
Total params: 16,812,554
Trainable params: 2,097,866
Non-trainable params: 14,714,688
_________________________________

###compile & fit transfer model


In [5]:
optimizer = Adam(lr=0.2e-3, beta_1=0.9, beta_2=0.999, amsgrad=False)
early_stopping = EarlyStopping(restore_best_weights=True)

transfer_model.compile(optimizer,loss='binary_crossentropy',metrics=["accuracy"])

checkpoint = ModelCheckpoint("VGG16_16_epochs_256_imres_Model.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=3, verbose=1, mode='auto')
hist = transfer_model.fit_generator(generator=train_data, validation_data= validation_data ,epochs=16,callbacks=[checkpoint,early]) 


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Epoch 1/30
  1/150 [..............................] - ETA: 17:00 - loss: 0.3674 - accuracy: 0.8969

In [ ]:
transfer_model.save('VGG16_16_epochs_256_imres_Model.h5')


In [ ]:
acc = history.history["accuracy"]
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,max(plt.ylim())])
plt.title('Training and Validation Loss')
plt.show()